In [1]:
import tensorflow as tf
from timeit import default_timer as timer
%run SimulatorIO.ipynb
%run NeuralNetGenerator.ipynb

In [5]:
class GameFunctions:
    def load():
        Names.loadNames()
        Databases.load()
    def getFirstMoveStr(action, player):
        teamList = [i + 1 for i in range(6)]
        pkmn = action - 3
        teamList.remove(pkmn)
        teamList.insert(0, pkmn)
        teamList = [str(num) for num in teamList]
        teamStr = ''.join(teamList)
        start = ''
        if player == 1:
            start = ">p1"
        else:
            start = ">p2"
        return start + " team " + teamStr
    def nnDoFirstMove(p, nn, inptTensor, player, playerDict, canonicalNames):
        playerDict["forceSwitch"] = True
        playerDict["active"] = ''
        action_logits = nn(inptTensor)[0]
        action_probs = ValidationFunctions.processActionProbs(action_logits, playerDict, canonicalNames)
        print(action_probs)
        action = selectAction(action_probs)
        print(action)
        ProcessFunctions.sendString(p, GameFunctions.getFirstMoveStr(int(action.numpy()), player))
        return action_probs[0, action]
    def nnDoMove(p, nn, inptTensor, player, playerDict, canonicalNames):
        action_logits = nn(inptTensor)[0]
        action_probs = ValidationFunctions.processActionProbs(action_logits, playerDict, canonicalNames)
        print(action_probs)
        print(f"player {player} has action probs of {action_probs}")
        action = selectAction(action_probs)
        ProcessFunctions.doMove(p, player, int(action.numpy()), canonicalNames, list(playerDict['names'].keys()))
        return action_probs[0, action]
    def selectMaxDamageMove(p, player, playerDict, canonicalNames):
        invalidMoves = ValidationFunctions.getInvalidMoves(playerDict, canonicalNames)
        maxMove = ValidationFunctions.getMaxDamageMove(playerDict)
        if maxMove not in invalidMoves and maxMove is not None:
            ProcessFunctions.doMove(p, player, maxMove, canonicalNames, list(playerDict['names'].keys()))
        else:
            while True:
                i = random.randint(0, 9) 
                if i not in invalidMoves:
                    ProcessFunctions.doMove(p, player, i, canonicalNames, list(playerDict['names'].keys()))
                    break
    def getMaxDamageTeam():
        with open("MaxTeam.txt", "r") as teamFile:
            return teamFile.read()
    def createReturnDict(winner, p1action_probs, p1values, p1states, p2action_probs, p2values, p2states):
        returnDict = {}
        returnDict['winner'] = winner
        returnDict['p1'] = {}
        returnDict['p1']['probs'] = tf.expand_dims(p1action_probs.stack(), 1)
        returnDict['p1']['values'] = tf.expand_dims(p1values.stack(), 1)
        returnDict['p1']['states'] = p1states
        returnDict['p2'] = {}
        returnDict['p2']['probs'] = tf.expand_dims(p2action_probs.stack(), 1)
        returnDict['p2']['values'] = tf.expand_dims(p2values.stack(), 1)
        returnDict['p2']['states'] = p2states
        return returnDict
    def createHalfReturnDict(winner, p1action_probs, p1values, p1states):
        returnDict = {}
        returnDict['winner'] = winner
        returnDict['p1'] = {}
        returnDict['p1']['probs'] = tf.expand_dims(p1action_probs.stack(), 1)
        returnDict['p1']['values'] = tf.expand_dims(p1values.stack(), 1)
        returnDict['p1']['states'] = p1states
        return returnDict
    def testAgainstMaxDamage(playerObject1, name1 = '', name2= 'Maximillian'):
        p1action_probs = tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True)
        p1values = tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True)
        p1states = tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True)

        p, gameEffectDict, p1Dict, p2Dict, name1, name2 = ProcessFunctions.startGameAndGetDicts(name1 = name1, name2 = name2)
        p1canonicalNames = ScrapingFunctions.getCanonicalNames(p1Dict)
        p2canonicalNames = ScrapingFunctions.getCanonicalNames(p2Dict)

        inptTensorP1 = NeuralNetInputFunctions.getNNinput(gameEffectDict, p1Dict, p2Dict, p1canonicalNames, p2canonicalNames, 1)
        p1index = 0
        p1action_probs = p1action_probs.write(p1index, GameFunctions.nnDoFirstMove(p, playerObject1.net, inptTensorP1, 1, p1Dict, p1canonicalNames))

        ProcessFunctions.sendString(p, GameFunctions.getFirstMoveStr(5, 2), 2)

        winner = ''
        while winner == '':
            p1canonicalNames = ScrapingFunctions.getCanonicalNames(p1Dict) # get canonical names
            p2canonicalNames = ScrapingFunctions.getCanonicalNames(p2Dict)
            output = ProcessFunctions.getTurnOutput(p)
            if ProcessFunctions.testLinesForWin(output, name1, name2) != 0:
                p.close()
                p1states.mark_used()
                p1action_probs.mark_used()
                p1values.mark_used()
                winner = ProcessFunctions.testLinesForWin(output, name1, name2)
                return GameFunctions.createHalfReturnDict(winner, p1action_probs, p1values, p1states)
            try:   
                gameEffectDict, p1Dict, p2Dict = ScrapingFunctions.getDicts(output, gameEffectDict)
            except AssertionError:
                print(output)
                assert(False)
            if 'wait' not in p1Dict:
                inptTensorP1 = NeuralNetInputFunctions.getNNinput(gameEffectDict, p1Dict, p2Dict, p1canonicalNames, p2canonicalNames, 1)
                p1states = p1states.write(p1index, inptTensorP1)
                p1action_probs = p1action_probs.write(p1index, GameFunctions.nnDoMove(p, playerObject1.net, inptTensorP1, 1, p1Dict, p1canonicalNames))
                value = playerObject1.critic(tf.reshape(inptTensorP1, [1, SEQUENTIALINPUTSHAPE[0]]))
                p1values = p1values.write(p1index, value)
                p1index += 1
            if 'wait' not in p2Dict:
                GameFunctions.selectMaxDamageMove(p, 2, p2Dict, p2canonicalNames)
    def runGame(playerObject1, playerObject2, name1 = '', name2 = ''):
        
        p1action_probs = tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True)
        p1values = tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True)
        p1states = tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True)
        
        p2action_probs = tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True)
        p2values = tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True)
        p2states = tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True)
        
        p, gameEffectDict, p1Dict, p2Dict, name1, name2 = ProcessFunctions.startGameAndGetDicts(name1 = name1, name2 = name2)
        p1canonicalNames = ScrapingFunctions.getCanonicalNames(p1Dict) # get canonical names
        p2canonicalNames = ScrapingFunctions.getCanonicalNames(p2Dict)
        
        inptTensorP1 = NeuralNetInputFunctions.getNNinput(gameEffectDict, p1Dict, p2Dict, p1canonicalNames, p2canonicalNames, 1)
        inptTensorP2 = NeuralNetInputFunctions.getNNinput(gameEffectDict, p1Dict, p2Dict, p1canonicalNames, p2canonicalNames, 2)
        
        p1index = 0
        p2index = 0
        p1action_probs = p1action_probs.write(p1index, GameFunctions.nnDoFirstMove(p, playerObject1.net, inptTensorP1, 1, p1Dict, p1canonicalNames))
        p2action_probs = p2action_probs.write(p2index, GameFunctions.nnDoFirstMove(p, playerObject2.net, inptTensorP2, 2, p2Dict, p2canonicalNames))
        p1values = p1values.write(p1index, playerObject1.critic(tf.reshape(inptTensorP1, [1, SEQUENTIALINPUTSHAPE[0]])))
        p2values = p2values.write(p2index, playerObject2.critic(tf.reshape(inptTensorP2, [1, SEQUENTIALINPUTSHAPE[0]])))

        winner = ''
        
        while winner == '':
            p1canonicalNames = ScrapingFunctions.getCanonicalNames(p1Dict) # get canonical names
            p2canonicalNames = ScrapingFunctions.getCanonicalNames(p2Dict)
            output = ProcessFunctions.getTurnOutput(p)
            if ProcessFunctions.testLinesForWin(output, name1, name2) != 0:
                p.close()
                winner = ProcessFunctions.testLinesForWin(output, name1, name2)
                p1states.mark_used()
                p2states.mark_used()
                return GameFunctions.createReturnDict(winner, p1action_probs, p1values, p1states, 
                                                      p2action_probs, p2values, p2states)
            try:   
                gameEffectDict, p1Dict, p2Dict = ScrapingFunctions.getDicts(output, gameEffectDict)
            except AssertionError:
                print(output)
                assert(False)
            if 'wait' not in p1Dict:
                inptTensorP1 = NeuralNetInputFunctions.getNNinput(gameEffectDict, p1Dict, p2Dict, p1canonicalNames, p2canonicalNames, 1)
                p1states = p1states.write(p1index, inptTensorP1)
                p1action_probs = p1action_probs.write(p1index, GameFunctions.nnDoMove(p, playerObject1.net, inptTensorP1, 1, p1Dict, p1canonicalNames))
                value = playerObject1.critic(tf.reshape(inptTensorP1, [1, SEQUENTIALINPUTSHAPE[0]]))
                p1values = p1values.write(p1index, value)
                p1index += 1
            if 'wait' not in p2Dict:
                inptTensorP2 = NeuralNetInputFunctions.getNNinput(gameEffectDict, p1Dict, p2Dict, p1canonicalNames, p2canonicalNames, 2)
                p2states = p2states.write(p2index, inptTensorP2)
                p2action_probs = p2action_probs.write(p2index, GameFunctions.nnDoMove(p, playerObject2.net, inptTensorP2, 2, p2Dict, p2canonicalNames))
                value = playerObject2.critic(tf.reshape(inptTensorP2, [1, SEQUENTIALINPUTSHAPE[0]]))
                p2values = p2values.write(p2index, value)
                p2index += 1